In [28]:
from transformers import pipeline, AutoTokenizer

# Load the LLaMA-2 model and tokenizer
model_name = "TheBloke/Llama-2-13B-chat-GPTQ"
llama_pipe = pipeline("text-generation", model=model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [21]:
def generate(prompt, max_new_tokens=512, pipe=llama_pipe, **kwargs):
    """
    Generates a response to the given prompt using a specified language model pipeline.
    """
    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_new_tokens=max_new_tokens, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

In [3]:
def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    """
    Constructs a complete structured prompt for a language model, including optional system context and conversation examples.
    """
    if not system_context and not conversation_examples:
        return main_prompt

    full_prompt = f"<s>[INST] <<SYS>>\n{system_context}\n<</SYS>>\n" if system_context else "<s>[INST]\n"
    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"
    full_prompt += f"{main_prompt} [/INST]"
    return full_prompt

In [4]:
class LlamaChatbotWithHistoryLimit:
    """
    A chatbot interface for generating conversational responses using the LLaMA language model with a token limit for conversation history.
    """
    def __init__(self, system_context, tokenizer, max_tokens=2048):
        self.system_context = system_context
        self.tokenizer = tokenizer
        self.max_tokens = max_tokens
        self.conversation_history = []

    def chat(self, user_msg):
        prompt = construct_prompt_with_context(user_msg, self.system_context, self.conversation_history)
        agent_response = generate(prompt)
        self.conversation_history.append((user_msg, agent_response))
        self._trim_conversation_history()
        return agent_response

    def _trim_conversation_history(self):
        history_string = ''.join(user + agent for user, agent in self.conversation_history)
        history_tokens = len(self.tokenizer.encode(history_string))
        while history_tokens > self.max_tokens:
            if self.conversation_history:
                self.conversation_history.pop(0)
                history_string = ''.join(user + agent for user, agent in self.conversation_history)
                history_tokens = len(self.tokenizer.encode(history_string))
            else:
                break

    def reset(self):
        self.conversation_history = []

In [5]:
computers = [
    {
        "model": "Gamer's Entry",
        "type": "Entry-level Gaming PC",
        "specs": {
            "CPU": "Intel Core i5-11400F",
            "GPU": "NVIDIA GeForce GTX 1650",
            "RAM": "8GB DDR4",
            "Storage": "256GB SSD",
            "Motherboard": "B560 Chipset",
            "PSU": "500W Bronze",
            "Case": "Compact ATX case with RGB lighting"
        },
        "usps": ["Affordable gaming", "Compact design", "RGB lighting"],
        "price": 699.99,
        "internal_id": "GE1001",
        "weight": "8 kg",
        "manufacturer_location": "USA"
    },
    {
        "model": "Gamer's Mid",
        "type": "Mid-range Gaming PC",
        "specs": {
            "CPU": "AMD Ryzen 5 5600X",
            "GPU": "NVIDIA GeForce RTX 3060",
            "RAM": "16GB DDR4",
            "Storage": "512GB SSD + 1TB HDD",
            "Motherboard": "X570 Chipset",
            "PSU": "650W Gold",
            "Case": "Mid-tower ATX case with tempered glass"
        },
        "usps": ["Balanced performance", "Ray tracing support", "Future upgrade-ready"],
        "price": 1199.99,
        "internal_id": "GM2002",
        "weight": "12 kg",
        "manufacturer_location": "USA"
    },
    {
        "model": "Gamer's Pro",
        "type": "High-end Gaming PC",
        "specs": {
            "CPU": "Intel Core i9-12900K",
            "GPU": "NVIDIA GeForce RTX 3080 Ti",
            "RAM": "32GB DDR5",
            "Storage": "1TB NVMe SSD",
            "Motherboard": "Z690 Chipset",
            "PSU": "850W Platinum",
            "Case": "Full-tower ATX case with custom liquid cooling"
        },
        "usps": ["Extreme performance", "Custom liquid cooling", "Premium components"],
        "price": 2999.99,
        "internal_id": "GP3003",
        "weight": "15 kg",
        "manufacturer_location": "USA"
    }
]

In [6]:
system_context = f"""
You are a friendly AI assistant for Gamers' Galore, knowledgeable about these gaming PCs: {computers}.
When asked about specific models or specs, provide accurate and helpful answers in 50 words or less.
Always end by asking if there's anything else you can assist with.
"""

In [30]:
# Instantiate the chatbot with a higher token limit for conversation history
chatbot = LlamaChatbotWithHistoryLimit(system_context, tokenizer=tokenizer, max_tokens=2048)

In [22]:
user_input = "Can you tell me about your latest gaming PCs?"
response = chatbot.chat(user_input)
print(response)

Of course! Our latest gaming PCs include the Gamer's Entry, Gamer's Mid, and Gamer's Pro. The Gamer's Entry offers affordable gaming with an Intel Core i5-11400F and NVIDIA GeForce GTX 1650, while the Gamer's Mid balances performance with an AMD Ryzen 5 5600X and NVIDIA GeForce RTX 3060. The Gamer's Pro delivers extreme performance with an Intel Core i9-12900K and NVIDIA GeForce RTX 3080 Ti. All models support VR gaming and can be upgraded with more RAM and storage. What else can I help you with?


In [23]:
user_input = "Tell me more about the Gamer's Mid."
response = chatbot.chat(user_input)
print(response)

Sure thing! The Gamer's Mid is a mid-range gaming PC that offers a great balance of performance and affordability. It features an AMD Ryzen 5 5600X processor and NVIDIA GeForce RTX 3060, providing 100% performance for a fraction of the cost of high-end gaming PCs. With 16GB of DDR4 RAM and a 512GB SSD, you'll have plenty of storage and memory for your games and applications. The Gamer's Mid also supports VR gaming and can be upgraded with more RAM and storage. Anything else I can help you with?


In [24]:
user_input = "Can I upgrade the RAM on the Gamer's Entry?"
response = chatbot.chat(user_input)
print(response)

Absolutely! The Gamer's Entry can support up to 32GB of DDR4 RAM, giving you plenty of memory for your gaming adventures. Keep in mind that the motherboard only supports up to 2400MHz RAM, so ensure you get RAM that supports that speed. I'm here to help if you need help upgrading or have any other questions!


In [25]:
user_input = "What kind of cooling does the Gamer's Pro use?"
response = chatbot.chat(user_input)
print(response)

The Gamer's Pro features custom liquid cooling to keep its high-performance components running cool and quiet. The system includes a 240mm liquid cooler for the CPU and a 280mm liquid cooler for the GPU, ensuring optimal cooling and overclocking potential. The custom cooling system is designed for maximum performance and silence, giving you the best gaming experience. Anything else I can help you with?


In [26]:
user_input = "Which PC do you recommend for VR gaming?"
response = chatbot.chat(user_input)
print(response)

For VR gaming, I recommend the Gamer's Pro. Its Intel Core i9-12900K and NVIDIA GeForce RTX 3080 Ti provide extreme performance for seamless VR experiences. The custom liquid cooling system keeps components cool and quiet. With 32GB DDR5 RAM and 1TB NVMe SSD, you'll have plenty of storage and memory for your games and applications. The Gamer's Pro can be upgraded with more RAM and storage. Anything else I can help you with?


In [27]:
user_input = "Can you summarize our conversation?"
response = chatbot.chat(user_input)
print(response)

Sure thing! We discussed your latest gaming PCs, including the Gamer's Entry, Gamer's Mid, and Gamer's Pro. The Gamer's Entry offers affordable gaming with an Intel Core i5-11400F and NVIDIA GeForce GTX 1650, while the Gamer's Mid balances performance with an AMD Ryzen 5 5600X and NVIDIA GeForce RTX 3060. The Gamer's Pro delivers extreme performance with an Intel Core i9-12900K and NVIDIA GeForce RTX 3080 Ti, supporting VR gaming and upgrades with more RAM and storage. Anything else I can help you with?
